In [1]:
import os
import json
from typing import Dict
import plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

In [2]:
num_days_hr = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/My_Thesis/chase_conference/CarePortal_HR.csv"


In [3]:
df = pd.read_csv(num_days_hr)

In [4]:
df.shape

(116, 2)

In [5]:
df.head()

,PatientID_HR,N_Days_Hr
0,21,8
1,22,20
2,23,12
3,24,20
4,25,17


In [18]:

above_10_DAYS = df[df["N_Days_Hr"] >= 10]

In [19]:
above_10_DAYS.shape

(58, 2)

In [8]:
large_rockwell_template = dict(
    layout=go.Layout(title_font=dict(family="Rockwell")))

In [10]:
fig = make_subplots(rows=1, cols=1,
        subplot_titles=("Total Number of Days for all Patients In The Study (N=11)"))



In [94]:
fig = px.bar(df,x=df.index,y=df["N_Days_Hr"].sort_values(ascending=False))#text="N_Days_Hr"
fig.add_hline(y=10)

fig.update_yaxes(title_text="Number of Days Completed",title_font_family="IBM Plex San", row=1, col=1)
fig.update_xaxes(title_text="Participant ID", title_font_family="IBM Plex San",row=1, col=1) #  tickangle=35,tickvals=df.index,showticklabels=True
fig.update_traces(marker_color='rgb(24, 128, 139)', marker_line_color='rgb(8,48,107)')
fig.update_layout(title="Participant Adherence",template = large_rockwell_template)



## Greater than eq to 10 days only

In [29]:
y = df[df["N_Days_Hr"] >= 10]["N_Days_Hr"]
x = []
for index, value in enumerate(y):
    x.append(index)
df_above10 = pd.DataFrame({"PID":x,
             "Num_HR_Days":y.sort_values(ascending=False)
             })

In [30]:
fig = px.bar(df_above10,x="PID",y="Num_HR_Days",text="Num_HR_Days")
fig.update_yaxes(title_text="Number of Days Completed",title_font_family="IBM Plex San", row=1, col=1)
fig.update_xaxes(title_text="Participant ID", title_font_family="IBM Plex San",row=1, col=1)
fig.update_traces(marker_color='rgb(24, 128, 139)', marker_line_color='rgb(8,48,107)')
fig.update_layout(template = large_rockwell_template)


# Duplicate Timestamps.